# Evaluate model to recognize sidewalks. 



### Initialize and load model

In [1]:
# Set location of test data and saved models on google drive
#data_path = "LA_test1_rev2"
data_path = "LA_train2"
#image_dir = 'drive/My Drive/CoLab_Data/Hollywood/'
image_dir = 'drive/My Drive/CoLab_Data/LosAngeles/'
saved_model_dir = 'drive/My Drive/CoLab_Data/save/'


# import dependencies
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt
%tensorflow_version 2.x
import tensorflow as tf
from google.colab import files
from google.colab import drive
from sklearn.metrics import classification_report, confusion_matrix

# connect to Google drive
drive.mount('/content/drive')

# unzip training images to local colab drive
unzip_path = '%s%s.zip' % (image_dir, data_path)
print(unzip_path)
os.system('unzip "%s%s.zip"' % (image_dir, data_path))
#os.system('rm %s/*.txt' % data_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
drive/My Drive/CoLab_Data/LosAngeles/LA_train2.zip


0

In [25]:
# Load model directly 
model_name = 'mobileV2_LA_0403_rev1'
model = tf.keras.models.load_model(saved_model_dir + model_name + '.h5')

In [48]:
# load different checkpoint for model
checkpoint = 'cp-0020.ckpt'
checkpoint_path = 'drive/My Drive/CoLab_Data/save/mobileV2_LA_0409'
model.load_weights(checkpoint_path + '/' + checkpoint)

In [49]:
# run inference on images

# define image size and classes
IMAGE_SIZE = 224
BATCH_SIZE = 32
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
CLASS_NAMES=('bike_lane', 'crosswalk', 'side','street')

BATCH_SIZE=1000

#eval_folder = "train"
eval_folder = "val"
data_path_eval = data_path + '/' + eval_folder
print(data_path_eval)

# open file for inference results
outFile = image_dir + data_path + eval_folder + '_out_data.txt'
outf = {'frame' : 0, 'label' : 1, 'class' : 2, 'swFlag' : 8, 'pos' : 9, 'vel' : 12}
outfile_format = '%5s,%d,' + 6 * '%3.2f,' + '%d,' + 3 * '%14.9f,' + 3 * '%5.2f,' + '\n'
fid = open(outFile, "w")

# create image generator
datagen_val = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
gen = datagen_val.flow_from_directory(
    data_path_eval,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle = False)
num_images = gen.n

# run inference on images and write results to file
ix=0
lb = []
y_pred = []
for n in range(num_images // BATCH_SIZE + 1):
  print(n * BATCH_SIZE)
  image_batch, label_batch = next(gen)
  label_batch = np.argmax(label_batch, axis=1)

  Y_pred = model.predict(image_batch)
  y_pred=np.concatenate((y_pred,np.argmax(Y_pred, axis=1)))
  lb=np.concatenate((lb, label_batch))

  for n in range(len(label_batch)):
    fn = gen.filenames[n+ix]
    img_num = fn[-19:-13] + fn[-12:-10] + fn[-9:-4]
    fid.write(outfile_format % (img_num, y_pred[ix+n], Y_pred[n][0], Y_pred[n][1],
              Y_pred[n][2], Y_pred[n][3], 0, 0, 0, 0, 0, 0, 0, 0, 0))
  ix += BATCH_SIZE

fid.close()

# create Confusion Matrix
print('Confusion Matrix')
cm = confusion_matrix(lb, y_pred)
cm_norm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=3)
print(cm_norm)

# Calculate summary numbers for side and street
#print('False positive: %5.1f%%' % (100 * cm_norm[0,1]))
#print('False negative: %5.1f%%' % (100 * cm_norm[1,0]))

# Calculate summary numbers for side, street, and crosswalk
print('False positive: %5.2f%%' % (100 * (cm[0,1] + cm[0,2] + cm[3,1] + cm[3,2]) / np.sum(cm)))
print('False negative: %5.2f%%' % (100 * (cm[1,0] + cm[1,3] + cm[2,0] + cm[2,3]) / np.sum(cm)))



LA_train2/val
Found 9338 images belonging to 4 classes.
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
Confusion Matrix
[[0.951 0.003 0.007 0.039]
 [0.002 0.975 0.007 0.016]
 [0.002 0.01  0.969 0.019]
 [0.009 0.012 0.015 0.964]]
False positive:  1.50%
False negative:  0.86%


In [8]:
# Setup to plot incorrect images

batch_num = 0
BATCH_SIZE = 1000
gen = datagen_val.flow_from_directory(
    data_path_eval,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle = False)

def show_batch(image_batch, label_batch):
  plt.figure(figsize=(40,40))
  plt_ix = 1
  for n in range(len(y_pred)):
      if label_batch[n] != y_pred[n] and plt_ix<=36:
        if label_batch[n] + y_pred[n] == 3:
          continue 
        ax = plt.subplot(6,6, plt_ix)
        plt_ix +=1
        plt.imshow(image_batch[n])
        plt.title(gen.filenames[batch_num * BATCH_SIZE + n] + '->' + CLASS_NAMES[y_pred[n]])
        plt.axis('off')

Found 9338 images belonging to 4 classes.


In [0]:
# plot incorrect images from next batch
image_batch, label_batch = next(gen)

label_batch = np.argmax(label_batch, axis=1)
Y_pred = model.predict(image_batch)
y_pred=np.argmax(Y_pred, axis=1)

show_batch(image_batch, label_batch)
batch_num += 1
print(batch_num)

In [0]:
# download results
files.download(outFile)

In [0]:
 # Convert to TFlite, save and download results

model_name = 'mobileV2_LA_0407'

 # Save model 
model.save(model_name + '.h5')
model.save(saved_model_dir + model_name + '.h5')
tf.saved_model.save(model, model_name)

# convert to tflite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# save TFlite model to Google drive and download folder on local drive 
with open(saved_model_dir + model_name + '.tflite', 'wb') as f:
  f.write(tflite_model)
with open(model_name + '.tflite', 'wb') as f:
  f.write(tflite_model)
files.download(model_name + '.tflite')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: mobileV2_LA_0407/assets
